# Fine-Tuning GPT-2 on recipe dataset
This notebook fine-tunes the language generation model GPT-2 to a recipe dataset. The goal is to input ingredients and let the model generate the recipe instructions.

The code is adapted from [Max Woolf](http://minimaxir.com)'s [notebook Train a GPT-2 Text-Generating Model w/ GPU For Free](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) (as at November 10th, 2019).

In [ ]:
# Accessing the downloaded dataset from my Drive and importing gpt_2_simple
%tensorflow_version 2.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import drive

#Mounting Collab to my GDrive
drive.mount('/content/drive/', force_remount=True)
#gpt2.mount_gdrive()

# Accessing the project file with the model and data
!unzip -n /content/drive/MyDrive/ecs271finalproject.zip

# List of files in my Google Drive leading to the project file
!ls /content/ecs271finalproject

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Mounted at /content/drive/
Archive:  /content/drive/MyDrive/ecs271finalproject.zip
checkpoint			    README.md		       RecipeNLG_dataset.csv
df_tokenized_with_2_separators.txt  recipe_dataset_builder.py  samples
df_tokenized_with_separator.txt     recipe-generator	       smaller_dataset.csv
models				    recipe_generator.py


In [ ]:
# Downloading GPT-2 Model
#Already downloaded it, don't need to run and resave downloaded "model" file again. Will run it if i want a larger model
#gpt2.download_gpt2(model_name="124M")

## Uploading the Files from my Google Drive

This cell will save the path towards the directory that leads to my project folder **(saved in my Google Drive that is also shared with you)** containing the model, dataset, and other important data.

In [ ]:
file_path = '/content/ecs271finalproject'
dataset_file_path = file_path + "/df_tokenized_with_separator.txt" #Parsed Dataset you're using for you GPT
model_file_path = file_path + "/models/124M"

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies.

In [ ]:
""" Already Ran it. Took approximately 3 hours. Lowest average error reached: 1.32.
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=dataset_file_path,
              model_name = model_file_path,
              batch_size=4,
              steps=-1,
              restore_from='fresh',
              run_name='training_stil_step_-1',
              print_every=10,
              sample_every=200,
              save_every=500
              )
  """

" Runs for approximately 3 hours. Lowest average error reached: 1.32.\nsess = gpt2.start_tf_sess()\n\ngpt2.finetune(sess,\n              dataset=dataset_file_path,\n              model_name = model_file_path,\n              batch_size=4,\n              steps=-1,\n              restore_from='fresh',\n              run_name='training_stil_step_-1',\n              print_every=10,\n              sample_every=200,\n              save_every=500\n              )\n  "

In [ ]:
# Copied the newly trained model's checkpoint folder to my Google Drive.
#gpt2.copy_checkpoint_to_gdrive(run_name='run3')

Done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on the retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
#gpt2.copy_checkpoint_from_gdrive(run_name='run3')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
#Since I already have run1 created, I will have the file path for that here:
run_file_path = file_path + "/checkpoint/run1"

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_file_path)

Loading checkpoint /content/ecs271finalproject/checkpoint/run1/model-100


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name=run_file_path)

''Don Juan''S Keep''S''Applesauce''' '["apple juice", "cinnamon", "sugar", "flour", "lemon juice", "soda", "egg", "vanilla", "salt", "egg", "vanilla", "sugar", "vanilla"]' '["Mix together the apple juice and cinnamon. Add sugar and beat until stiff. Add egg and beat until well blended. Add vanilla and beat well. Add meringue mixture and beat well.", "Add honey and sweetener.", "Pour into cooled applesauce and chill.", "Bake in preheated oven at 325\u00b0 for 30 to 35 minutes."]' </>separator</>
'Dumpling Soup' '["ground beef", "salt", "pepper", "celery", "pepper soup", "Cheddar cheese"]' '["Cook beef, onion and celery in salted boiling water.", "Add soup and Cheddar cheese.", "Garnish with chopped celery and chopped pepper.", "Mix together cheese and chopped beef.", "Pour over beef.", "Pour drained cheese and mixture into a moldy and floured pan.", "Bake at 300\u00b0 for 30 minutes.", "Frost immediately."]' </>separator</>
'Turkey Casserole' '["turkey", "butter", "onion", "cornstarch",

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
x = gpt2.generate(sess,
              temperature=0.75,
              prefix="['bell peppers', 'rice', 'chili powder', 'potatos', 'mushroom']",
              nsamples=1,
              run_name=run_file_path,
              truncate='</>separator</>',
              include_prefix=False
              )

print(x)

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint/run1/encoder.json'

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.